In [1]:
import numpy as np
import pandas as pd

from util_io import init, finalize, dump_conf
from util_cost import cal_total, n_people, family_id_choice_to_pref_cost
from util_cost import choices as family_pref
from util_check import deep_check

In [2]:
# constants #
N_families = 5000
N_days = 100
# constants #

# params #
path_init_conf =     '../output/m08-improved.csv'
path_dump_improved = '../output/m10-improved.csv' # lowest cost

In [3]:
assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
etotal_low = cal_total(assigned_day, occupancy)
print('Init config cost:', etotal_low)

Read initial configs...
Read config completed.
Init config cost: 73620.36100267638


In [4]:
len(assigned_day)

5000

## Ortools MIP

In [5]:
from ortools.linear_solver import pywraplp

In [6]:
# solver = pywraplp.Solver('', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
solver = pywraplp.Solver('', pywraplp.Solver.CPLEX_MIXED_INTEGER_PROGRAMMING)

AttributeError: type object 'Solver' has no attribute 'CPLEX_MIXED_INTEGER_PROGRAMMING'

In [1]:
#solver.SetSolverSpecificParametersAsString('threads')

In [13]:
solver.SetNumThreads(24)

True

In [8]:
families = range(N_families)
days = range(1, N_days + 1)

In [9]:
# Possible choice for the family
# last choice is any day that is not on the family's preferred days
N_choices = family_id_choice_to_pref_cost.shape[1]
N_choices

11

In [10]:
family_id_choice_to_pref_cost

array([[   0,   50,   86, ...,  544, 1440, 2236],
       [   0,   50,   86, ...,  544, 1440, 2236],
       [   0,   50,   77, ...,  508, 1205, 1802],
       ...,
       [   0,   50,  104, ...,  616, 1910, 3104],
       [   0,   50,   95, ...,  580, 1675, 2670],
       [   0,   50,   86, ...,  544, 1440, 2236]], dtype=int64)

## Variables

In [11]:
# Variables
# assignment matrix[family, pref_rank]
assignment_matrix = {}
for family in families:
    for c in range(N_choices):
        assignment_matrix[family, c] = solver.BoolVar('x[%i,%i]' % (family, c))

In [12]:
len(assignment_matrix)

55000

In [13]:
possible_family_sizes = np.unique(n_people)

In [14]:
N_min_people = 125
N_max_people = 300

In [15]:
# unpreferred_day_counts[day, size]
unpreferred_day_counts = {}
for day in days:
    for size in possible_family_sizes:
        ub = int(N_max_people / size)
        # ub = solver.infinity()
        unpreferred_day_counts[day, size] = solver.IntVar(0, ub, 'd[%i,%i]' % (day, size))

In [16]:
len(unpreferred_day_counts)

700

## Constraints

### constraint 1: each family only take one day (choice)

In [17]:
# Constraints
# constraint 1: each family only take one day (choice)
for family in families:
    solver.Add(solver.Sum([assignment_matrix[family, c]
                           for c in range(N_choices)]) == 1)

### constraint 2: each day can only have 125-300 people

In [18]:
# constraint 2: each day can only have 125-300 people

In [19]:
family_pref

array([[ 52,  38,  12, ...,  76,  10,  28],
       [ 26,   4,  82, ...,   6,  66,  61],
       [100,  54,  25, ...,  89,  80,  33],
       ...,
       [ 32,  66,  54, ...,  81,   3,   7],
       [ 67,  92,   4, ...,  12,  26,  70],
       [ 13,  11,  25, ...,  39,  18,  47]], dtype=int64)

In [20]:
N_family_pref = N_choices - 1
N_family_pref

10

In [21]:
# day to dictionary of families who choice this day with value as preference rank
days_family_prefered = [{} for day in range(N_days+1)]  # day = 0 should not be used

In [22]:
for family, pref in enumerate(family_pref):
    for pref_rank, day in enumerate(pref):
        days_family_prefered[day][family] = pref_rank

In [23]:
for day in days:
    # find those family who like this day
    family_prefered = days_family_prefered[day]
    solver.Add(
        solver.Sum(
            [assignment_matrix[family, pref_rank] * n_people[family] 
             for family, pref_rank in family_prefered.items()]
        )
        + solver.Sum(
            [unpreferred_day_counts[day, s] * s for s in possible_family_sizes]
        )
        <= 300,
        'day_upper_bound[%i]' % day
    )
    solver.Add(
        solver.Sum(
            [assignment_matrix[family, pref_rank] * n_people[family] 
             for family, pref_rank in family_prefered.items()]
        )
        + solver.Sum(
            [unpreferred_day_counts[day, s] * s for s in possible_family_sizes]
        )        
        >= 125,
        'day_lower_bound[%i]' % day
    )

### constraint 3: unpreferred day family count conservation for each family size

In [24]:
# constraint 3: unpreferred day family count conservation for each family size

In [25]:
family_size_to_family_ids = {
    size: np.where(n_people == size)[0] for size in possible_family_sizes
}

In [26]:
for size in possible_family_sizes:
    solver.Add(
        solver.Sum([assignment_matrix[family, N_choices - 1]
                    for family in family_size_to_family_ids[size]])
        == solver.Sum([unpreferred_day_counts[day, size] for day in days]),
        'unpreferred_day_counts[%i]' % size
    )

## Objective

In [27]:
# Objective - Preference cost only as approximation
solver.Minimize(
    solver.Sum([
        assignment_matrix[family, c] * family_id_choice_to_pref_cost[family, c]
        for family in families for c in range(N_choices)
    ])
)

xx = solver.constraints()[0]

[xx.GetCoefficient(assignment_matrix[1, c]) for c in range(N_choices)]

xx.name()

## Solve

In [28]:
len(solver.variables())

55700

In [ ]:
%%time
# Solve
sol = solver.Solve()

print('Total cost = ', solver.Objective().Value())

In [ ]:
print("Time = ", solver.WallTime(), " milliseconds")

In [ ]:
pywraplp.Solver.OPTIMAL

## Debug

In [29]:
# [
#     [assignment_matrix[family, c].solution_value() for c in range(N_choices)]
#     for family in range(10)
# ]        

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [30]:
# [
#     [unpreferred_day_counts[day, size].solution_value() for size in possible_family_sizes]
#     for day in range(1, 10)
# ]        

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]